<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-chatbot
Este el cuaderno final donde vamos a utilizar los modelos ya entrenados para obtener la respuesta de la maquina. Vamos a cargar todo desde drive porque fué ahí donde se guardaron tanto los modelos como los diccionarios para hacer el preprocesamiento. Además, vamos a unir todo dentro de una sola función para que sea mas sencillo utilizarlo.

In [1]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# librerias
import pickle
import random
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

## 1.Clasificador
### Cargar modelo
Aquí descomprimimos el archivo que contiene el modelo clasificador y cargamos el modelo con *tf.keras.models*.

In [3]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
   creating: clasificador/
  inflating: clasificador/saved_model.pb  
   creating: clasificador/assets/
   creating: clasificador/variables/
  inflating: clasificador/variables/variables.index  
  inflating: clasificador/variables/variables.data-00001-of-00002  
  inflating: clasificador/variables/variables.data-00000-of-00002  


In [4]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#modelo_clasificador.summary() #arquitectura del modelo

### Cargar para preprocesamiento
El modelo recibe las entradas codificadas, por ello, cargamos los trigramas y los bigramas para hacer el preprocesamiento de las entradas del usuario.

In [5]:
#trigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [6]:
#bigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

In [7]:
#label_tokenizer
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

### Función clasificadora
Aquí utilizamos lo que cargamos anteriormente para que todo lo relacionado al modelo clasificador se pueda llamar desde una sola función. Hemos hecho además la función *sequences_to_text* para volver de los valores numericos a la etiqueta en texto porque la función *sequences_to text* de *label_tokenizer* (creado con *Tokenizer* de *tensorflow.keras.preprocessing.text*) parece no funcionar correctamante, ya que con esa función se obtenian de vuelta solo tres de las cuatro etiquetas.

In [8]:
# funcion de frase tokenizada a palabras
reverse_word_map = dict(map(reversed, label_tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Busca palabras en el diccionario
    label = [reverse_word_map.get(i) for i in list_of_indices]
    return(label)

In [84]:
def clasifica (sentence):
  sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
  sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
  # preprocesamiento trigramas
  vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
  X3 = vectorizer3.fit_transform([sentence])
  # preprocesamiento bigramas
  vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
  X2 = vectorizer2.fit_transform([sentence])
  # concatenamos
  sequences_bt = np.concatenate((X3.toarray(), X2.toarray()), axis=1)
  # predicción
  pred = np.argmax(modelo_clasificador.predict(sequences_bt), axis=-1) #prediccion del modelo
  pred = sequence_to_text(pred) #de tokenizer a label
  pred = ''.join(pred)
  
  return (pred)

In [85]:
#ejemplo
sentence = "Would this be a good time to be honest?"
clasifica(sentence)

'marge_simpson'

In [89]:
#ejemplo
sentence = "I want to fly like a bird"
clasifica(sentence)

'homer_simpson'

In [96]:
#ejemplo
sentence = "Eat my shorts"
clasifica(sentence)

'homer_simpson'

In [95]:
#ejemplo
sentence = "We are going to have an excellent time"
clasifica(sentence)

'marge_simpson'

## 2.Generador
### Cargar modelo
Igual que con el modelo anterior, cargamos todo lo relacionado a este modelo desde drive, tanto el modelo como los diccionarios para el pre y post procesamiento. En este caso tenemos un modelo generador sencillo *modelo_generador*, y otro con un entrenamiento más avanzado *modelo_generador_pers*.

In [12]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip
   creating: generador/
  inflating: generador/saved_model.pb  
   creating: generador/assets/
   creating: generador/variables/
  inflating: generador/variables/variables.index  
  inflating: generador/variables/variables.data-00001-of-00002  
  inflating: generador/variables/variables.data-00000-of-00002  


In [13]:
modelo_generador = tf.keras.models.load_model('generador')
#modelo_clasificador.summary() #arquitectura del modelo

In [14]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador_pers.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador_pers.zip
   creating: generador_pers/
  inflating: generador_pers/saved_model.pb  
   creating: generador_pers/assets/
   creating: generador_pers/variables/
  inflating: generador_pers/variables/variables.index  
  inflating: generador_pers/variables/variables.data-00001-of-00002  
  inflating: generador_pers/variables/variables.data-00000-of-00002  


In [15]:
modelo_generador_pers = tf.keras.models.load_model('generador_pers')
#modelo_clasificador.summary() #arquitectura del modelo

### Cargar para preprocesamiento
Para que el modelo acepte la entretada necesitamos transformar la entrada en texto a numeros, para esto cargamos los diccionarios de carácteres a índices y viceversa, índices a carácteres.

In [16]:
# char a id
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/charxid.pickle', 'rb') as f:
  charxid = pickle.load(f)

In [17]:
#id a char
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/idxchar.pickle', 'rb') as f:
  idxchar = pickle.load(f)

### Función generadora
Al igual que en el modelo anterior, utilizamos todo lo relacionado con el modelo dentro de la función generadora. La función tiene el atributo *model* que permite seleccionar entre el modelo basico *modelo_generador* y el modelo con un entrenamiento azanzado *modelo_generador_pers*.

In [18]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(model,start_string, num_generate):
  #vectorizacion
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension
  
  #guardamos el texto predicho
  text_generated = []
  temperature = 1.0
  #tamaño del bache== 1

  model.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = model(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

In [122]:
#ejemplo
sentence = "Would this be a good time to be honest?"
generate_text(modelo_generador,sentence, 100)

'Would this be a good time to be honest? Whip?\r\nLisa Simpson: (ANNOYED NOISES) See this?... can you remember that rame to your friend?\r\nLisa'

In [123]:
#ejemplo
sentence = "Homer:Would this be a good time to be honest?"
generate_text(modelo_generador,sentence, 100)

"Homer:Would this be a good time to be honest?\r\nLisa Simpson: Bart, I got a new business one.\r\nHomer Simpson: No.\r\nHomer Simpson: I can't chink in"

In [124]:
#ejemplo
sentence = "Would this be a good time to be honest?"
generate_text(modelo_generador_pers,sentence, 100)

"Would this be a good time to be honest?\r\nMarge Simpson: Oh.\r\nHomer Simpson: (SWEETLY) No, thank you, sir, wait for Homer, they're traveling"

In [125]:
#ejemplo
sentence = "Homer:Would this be a good time to be honest?"
generate_text(modelo_generador_pers,sentence, 100)

"Homer:Would this be a good time to be honest?\r\nParson: Hey Bart, you could take my cigarettes. But I don't want you to be plunged. (STERN) CALE, "

## 3.Funciones para unir redes
La salida de la red clasificadora es una etiqueta, sin embargo esas etiquetas contienen guion bajo, para limpiar la etiqueta utilizamos la función *etiquet*, Luego, para unir la salida de la red clasificadora con la entrada del generador de texto, utilizamos la función *unión*, y para limpiar la salida del generador, usamos la función *output_ultimate* que está contenida dentro de *union*. Además, tenemos una lista de frases de respuesta que se eligen aleatoriamente en caso de que el modelo clasificador no sea capaz de generar una etiqueta, así la maquina entregara una respuesta aleatoria definida previamente en lugar de entrar a la red generadora.

In [126]:
def etiquet(pre_etiqueta):
  if pre_etiqueta != '<OOV>':
    pre_etiqueta = pre_etiqueta[:pre_etiqueta.index('_')].capitalize()
  return (pre_etiqueta)

In [127]:
def output_ultimate(line):
  if line.split('\n')[1] == ' ':
    line2=line.split('\n')[2]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)
  else:
    line2=line.split('\n')[1]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)

In [128]:
# frases de respuesta si no logra clasificar
frases_respuesta = ["Mmm, donuts.",
                      "Whatever, I'll be at Moe's.",
                      "Eat my shorts!",
                      "If anyone wants me, I'll be in my room.",
                      "Bye!",
                      "To start press any key. Where's the ANY key?",
                      "The lesson is, never try",
                      "I'm normally not a praying man, but if you're up there, please save me, Superman",
                      "You'll have to speak up I'm wearing a towel",
                      "I am so smart, S-M-R-T",
                      "You're making a scene",
                      "Can I have some money now?"]

In [129]:
def union(model, lst1, lst2):
  if lst1 != '<OOV>':
    final_list = lst1 + ": " + lst2 + "\n"
    final_list = final_list.split('\n')[0]
    final_list = generate_text(model, final_list, 1000)
    final_list = output_ultimate(final_list)
  else:
    a = random.randint(0, 1)
    if a == 0:
      b = random.randint(0, len(frases_respuesta)) #elige una frase aleatoriamente
      final_list = frases_respuesta[b]
    else:
      final_list = lst2 + "\n"
      final_list = final_list.split('\n')[0]
      final_list = generate_text(model, final_list, 1000)
      final_list = output_ultimate(final_list)
  
  return (final_list)

In [130]:
#ejemplo
union(modelo_generador, 'Marge', "Oh, my homer, if you knew how much I love you")

 Why are you really won't go hat over this?


In [131]:
#ejemplo
union(modelo_generador_pers, 'Marge', "Oh, my homer, if you knew how much I love you")

 Yes.


In [134]:
#ejemplo: entra al generador
union(modelo_generador_pers,'<OOV>', "Oh, my homer, if you knew how much I love you")

 (FRMADING) With a New Western One as much big you don't exchange.


In [135]:
#ejemplo: elige una respuesta pre-establecida
union(modelo_generador_pers,'<OOV>', "Oh, my homer, if you knew how much I love you")

"To start press any key. Where's the ANY key?"

# 4.Pre chatbot
Ahora la función *prechatbot* une todo lo hecho en el repositorio y es posible usarla de una forma sencilla. ¿Como funciona internamente? Cuando el usuario pone una entrada, ésta entrada es transformada a un vector que indica la composición de la entrada con el vocabulario de la red clasificadora, la red clasificadora devuelve una etiqueta que conecta la forma de escribir de un ususario con el estilo de alguno de los personajes prinicipales de los simpsons, hacemos esto porque son los personajes principales quienes tienen mas interaciones con los demas personajes, si la etiqueta es valida, se une esa etiqueta con la entrada del usuario y se genera una respuesta, para generar la respuesta se utiliza bien o el modelo sencillo *modelo_generador* o el modelo con entrenamiento avanzado *modelo_generador_pers* que es definido por el usuario en la entrada, si la etiqueta no es valida, se devuelve aleatoriamente una frase predeterminada, o se genera una respuesta sin la etiqueta del personaje, tomando como entrada para el generador solo la entrada del usuario. 

In [136]:
def prechatbot(model,sentence):
  pre_etiqueta = clasifica(sentence)
  etiqueta = etiquet(pre_etiqueta)
  respuesta = union(model, etiqueta, sentence)
  return (respuesta)

In [138]:
#ejemplo
Usuario = "Eat my shorts"
print("modelo_generador")
prechatbot(modelo_generador, Usuario)

modelo_generador
 No, don't you please be going you?


In [139]:
print("modelo_generador_pers")
prechatbot(modelo_generador_pers, Usuario)

modelo_generador_pers
 I'm sorry, this just wasn't. Both?


In [ ]:
#ejemplo
Usuario = "Would this be a good time to be honest?"
print("modelo_generador")
prechatbot(modelo_generador, Usuario)

modelo_generador
 Well, have for there?


In [ ]:
print("modelo_generador_pers")
prechatbot(modelo_generador_pers, Usuario)

modelo_generador_pers
 (CONCENTLES) I don't like we hear therefeys for the next drivew beer.


In [ ]:
#ejemplo
Usuario = "Hello, how are you? Are you fine?"
print("modelo_generador")
prechatbot(modelo_generador, Usuario)

modelo_generador
 That's unie the cash. It's gotder and fly those plimy in the same it.


In [ ]:
print("modelo_generador_pers")
prechatbot(modelo_generador_pers, Usuario)

modelo_generador_pers
 I get off your imagination.


In [ ]:
#ejemplo
Usuario = "Manzana"
print("modelo_generador")
prechatbot(modelo_generador, Usuario)

modelo_generador


"Whatever, I'll be at Moe's."

In [ ]:
print("modelo_generador_pers")
prechatbot(modelo_generador_pers, Usuario)

modelo_generador_pers
 No!


In [ ]:
#ejemplo
Usuario = "Hello"
print("modelo_generador")
prechatbot(modelo_generador, Usuario)

modelo_generador
 Absolutely. You have to takep up him I'm are after well.


In [ ]:
print("modelo_generador_pers")
prechatbot(modelo_generador_pers, Usuario)

modelo_generador_pers
 I told that other seven gear. (THEN) Ux how much siren sideple dollacs. We just outlet you after seeing my teams lost.
